# Read dataset

In [1]:
import pandas as pd
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
A = pd.read_csv("C:/Users/Pawan/Downloads/training_set _loan.csv")

In [3]:
A.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


# Treat missing Data

In [4]:
from PM8_weekend import replacer
replacer(A)

# Define  X & Y

In [5]:
Y = A[["Loan_Status"]]
X = A.drop(["Loan_Status","Loan_ID"],axis = 1)

# Preprocessing

If categorical --> OHE --> 0/1

In [6]:
cat = []
con = []
for i in X.columns:
    if(X[i].dtypes=="object"):
        cat.append(i)
    else:
        con.append(i)

In [7]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
X1 = pd.DataFrame(mm.fit_transform(X[con]),columns=con)
X2 = pd.get_dummies(X[cat])
Xnew = X1.join(X2)

# Y-Label Encoder conversion

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y.Loan_Status = le.fit_transform(Y)

# Divide the data in training and testing set

In [9]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

In [35]:
Xnew.shape[1]

20

# Create a NN

In [11]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [12]:
nn = Sequential()
nn.add(Dense(200,input_dim=(Xnew.shape[1])))
nn.add(Dropout(0.1))

nn.add(Dense(40))
nn.add(Dropout(0.1))

nn.add(Dense(20))
nn.add(Dropout(0.1))

nn.add(Dense(1,activation="sigmoid"))


In [13]:
nn.compile(loss="binary_crossentropy",metrics="accuracy")
model = nn.fit(xtrain,ytrain,epochs=30,batch_size=100)

Epoch 1/30
5/5 [==============================] - 5s 62ms/step - loss: 0.6570 - accuracy: 0.6477
Epoch 2/30
5/5 [==============================] - 0s 9ms/step - loss: 0.5623 - accuracy: 0.7189
Epoch 3/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5440 - accuracy: 0.7515
Epoch 4/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5156 - accuracy: 0.7760
Epoch 5/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5095 - accuracy: 0.7923
Epoch 6/30
5/5 [==============================] - 0s 4ms/step - loss: 0.4950 - accuracy: 0.8024
Epoch 7/30
5/5 [==============================] - 0s 4ms/step - loss: 0.4850 - accuracy: 0.8086
Epoch 8/30
5/5 [==============================] - 0s 4ms/step - loss: 0.4825 - accuracy: 0.8004
Epoch 9/30
5/5 [==============================] - 0s 4ms/step - loss: 0.4776 - accuracy: 0.8065
Epoch 10/30
5/5 [==============================] - 0s 4ms/step - loss: 0.4814 - accuracy: 0.8024
Epoch 11/30
5/5 [=====================

# Keras Tuner:


1. Create a model builder function that returns a created NN
2. Create a Hyperband object(metrics for tuning)
3. Search function(To start tuning based on given parameters)

In [14]:
import keras_tuner as kt

In [15]:
def model_builder(hp):
    nn = Sequential()
    nn.add(Dense(200,input_dim=(Xnew.shape[1])))
    nn.add(Dropout(0.1))
    y = hp.Choice("units",values=[64,128,256,512])
    nn.add(Dense(units=y))
    x = hp.Choice("rate",values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
    nn.add(Dropout(rate=x))
    nn.add(Dense(20))
    nn.add(Dense(1,activation="sigmoid"))
    nn.compile(loss="binary_crossentropy",metrics="accuracy")
    return nn


In [16]:
tuner = kt.Hyperband(model_builder,
             objective="val_accuracy",
             max_epochs=10,
             directory="C:/users/Pawan/Downloads/NN",
             project_name="Tuner")

INFO:tensorflow:Reloading Oracle from existing project C:/users/Pawan/Downloads/NN\Tuner\oracle.json
INFO:tensorflow:Reloading Tuner from C:/users/Pawan/Downloads/NN\Tuner\tuner0.json


In [17]:
tuner.search(xtrain,ytrain,epochs=30,validation_split=0.2)

INFO:tensorflow:Oracle triggered exit


In [18]:
best = tuner.get_best_hyperparameters(num_trials=7)[0]

In [19]:
best.get("rate")  # Hyperparameter for DropOut

0.4

In [20]:
best.get("units") # Hyperparameter for Hidden layer size

512

# Create Predictions on Testing Data

In [21]:
B = pd.read_csv("C:/Users/Pawan/Downloads/testing_set _loan.csv")

In [22]:
B.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [23]:
replacer(B)

In [24]:
X = B.drop(["Loan_ID"],axis=1)

In [25]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
X1 = pd.DataFrame(mm.fit_transform(X[con]),columns=con)
X2 = pd.get_dummies(X[cat])
Xnew = X1.join(X2)

In [26]:
len(Xnew.columns)

20

In [27]:
nn = Sequential()
nn.add(Dense(200,input_dim=(Xnew.shape[1])))
nn.add(Dropout(0.1))

nn.add(Dense(512))
nn.add(Dropout(0.7))

nn.add(Dense(20))
nn.add(Dropout(0.1))

nn.add(Dense(1,activation="sigmoid"))

In [28]:
nn.compile(loss="binary_crossentropy",metrics="accuracy")
model = nn.fit(xtrain,ytrain,epochs=30,batch_size=100)

Epoch 1/30
5/5 [==============================] - 1s 12ms/step - loss: 0.7439 - accuracy: 0.6436
Epoch 2/30
5/5 [==============================] - 0s 9ms/step - loss: 0.5499 - accuracy: 0.7658
Epoch 3/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5261 - accuracy: 0.7719
Epoch 4/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5184 - accuracy: 0.7637
Epoch 5/30
5/5 [==============================] - 0s 2ms/step - loss: 0.4950 - accuracy: 0.7963
Epoch 6/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5068 - accuracy: 0.7780
Epoch 7/30
5/5 [==============================] - 0s 8ms/step - loss: 0.4910 - accuracy: 0.7882
Epoch 8/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5058 - accuracy: 0.7678
Epoch 9/30
5/5 [==============================] - 0s 6ms/step - loss: 0.4769 - accuracy: 0.8187
Epoch 10/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5174 - accuracy: 0.7943
Epoch 11/30
5/5 [=====================

In [29]:
Q = []
for i in nn.predict(Xnew):
    if(i[0] < 0.5):
        Q.append(0)
    else:
        Q.append(1)


In [30]:
Q = le.inverse_transform(Q)

In [31]:
W = B[["Loan_ID"]]
W['Loan_Status']=Q

In [32]:
B.shape

(367, 12)

In [33]:
len(Q)

367

In [34]:
W

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,N
...,...,...
362,LP002971,Y
363,LP002975,Y
364,LP002980,Y
365,LP002986,Y
